In [1]:
from support import solve_minimax_regret_optimization
import pandas as pd
import pickle as pkl
import numpy as np

In [2]:
# Define investment opportunities
investment_opportunities = [
    {'id': 'A', 'cost': 4, 'R_best': 15, 'R_med': 10, 'R_worst': 2, 'confidence': 0.9},
    {'id': 'B', 'cost': 3, 'R_best': 12, 'R_med': 8, 'R_worst': 1, 'confidence': 0.6},
    {'id': 'C', 'cost': 3, 'R_best': 9, 'R_med': 6, 'R_worst': 2, 'confidence': 0.80},
    {'id': 'D', 'cost': 2, 'R_best': 7, 'R_med': 5, 'R_worst': 3, 'confidence': 0.40},
    {'id': 'E', 'cost': 5, 'R_best': 18, 'R_med': 9, 'R_worst': 0, 'confidence': 0.50}, # Low confidence
]

In [3]:
investments = pd.DataFrame(investment_opportunities)
investments.set_index("id", inplace=True)
investments

,cost,R_best,R_med,R_worst,confidence
id,,,,,
A,4,15,10,2,0.9
B,3,12,8,1,0.6
C,3,9,6,2,0.8
D,2,7,5,3,0.4
E,5,18,9,0,0.5


In [4]:
# Define optimization parameters
BUDGET = 10
MIN_CONFIDENCE = 0.5 # Initiatives with confidence < 0.4 will be excluded
MIN_RETURN = 4 # Minimum aggregate R_worst for selected portfolio

# Solve the optimization problem
results = solve_minimax_regret_optimization(
    investment_opportunities,
    BUDGET,
    MIN_CONFIDENCE,
    MIN_RETURN)

# Print results
print("\n--- Optimization Results ---")
print(f"Solver Status: {results['status']}")
if results['status'] == 'Optimal':
    print(f"Minimized Maximum Regret: {results['min_max_regret']:.2f}")
    print(f"Selected Initiatives: {results['selected_initiatives']}")
    print(f"Total Cost of Selected Initiatives: {results['total_cost']:.2f}")
    print("\nEffective Returns of Selected Portfolio:")
    for s, r_val in results['total_actual_returns'].items():
        print(f"  Scenario {s}: {r_val:.2f}")

    print("\nOptimal Returns for Each Scenario (V_j_star):")
    for s, v_val in results['v_j_star'].items():
        print(f"  Scenario {s}: {v_val:.2f}")

    print("\nCalculated Regrets for the Selected Portfolio:")
    for s, r_val in results['regrets_for_selected_portfolio'].items():
        print(f"  Scenario {s}: {r_val:.2f}")

    # Verify the max regret from the selected portfolio matches the objective value
    if results['regrets_for_selected_portfolio']:
        actual_max_regret = max(results['regrets_for_selected_portfolio'].values())
        print(f"\nActual Max Regret (from selected portfolio): {actual_max_regret:.2f}")
        print(f"Does it match minimized max regret? {abs(actual_max_regret - results['min_max_regret']) < 1e-6}") # Check for floating point equality

else:
    print("Optimization did not find an optimal solution.")
    if 'v_j_star' in results and results['v_j_star']:
         print("\nOptimal Returns for Each Scenario (V_j_star):")
         for s, v_val in results['v_j_star'].items():
             print(f"  Scenario {s}: {v_val:.2f}")

--- Running Minimax Regret Optimization ---
Total Budget: 10
Min Confidence Threshold: 0.5
Min Portfolio Worst Return: 4

--- Calculating Optimal Scenario Returns (V_j_star) ---
  Scenario 'best': V_j_star = 28.90
  Scenario 'med': V_j_star = 19.60
  Scenario 'worst': V_j_star = 5.00

--- Formulating Minimax Regret Problem ---
Solving the main optimization problem...

--- Optimization Results ---
Solver Status: Optimal
Minimized Maximum Regret: 0.00
Selected Initiatives: ['A', 'B', 'C']
Total Cost of Selected Initiatives: 10.00

Effective Returns of Selected Portfolio:
  Scenario best: 28.90
  Scenario med: 19.60
  Scenario worst: 5.00

Optimal Returns for Each Scenario (V_j_star):
  Scenario best: 28.90
  Scenario med: 19.60
  Scenario worst: 5.00

Calculated Regrets for the Selected Portfolio:
  Scenario best: 0.00
  Scenario med: 0.00
  Scenario worst: 0.00

Actual Max Regret (from selected portfolio): 0.00
Does it match minimized max regret? True


In [5]:
#pkl.dump(results, open("regression_test.obj","wb"))

In [6]:
rslt = pkl.load(open('regression_test.obj', 'rb'))
np.testing.assert_equal(rslt, results)

#